<h1><div align="center">CUDA C/C++ 통합 메모리로 가속 애플리케이션 메모리 관리</div></h1>

![CUDA](./images/CUDA_Logo.jpg)

[*CUDA 모범 사례 가이드*](http://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations)는 이번 실습과 다른 CUDA 기본 실습 이후에 후속 학습하면 좋은 콘텐츠로, **A**ssess(평가), **P**arallelize(병렬화), **O**ptimize(최적화), **D**eploy(배포)의 약자인 **APOD** 디자인 주기를 권장합니다. 즉, APOD는 개발자가 가속 애플리케이션의 성능에 증분 개선을 적용하고 코드를 배포할 수 있는 반복적 디자인 프로세스를 처방하는 것입니다. 개발자가 더 유능한 CUDA 프로그래머가 되기 시작하면 가속 코드베이스에 더 고급 수준의 최적화 기법을 적용할 수 있습니다.

이 실습은 이러한 방식의 반복 개발을 지원할 것입니다. Nsight Systems 명령줄 도구인 **nsys**를 사용하여 애플리케이션의 성능을 정성적으로 측정하고, 최적화 기회를 파악하기 위해 새로운 기술을 학습하고 주기를 반복하기 전에 증분 개선을 적용합니다. 요점은 이 실습에서 학습하고 적용하는 많은 기법이 CUDA의 **통합 메모리**가 작동하는 방법의 세부 사항을 다루게 된다는 것입니다. 통합 메모리 동작을 이해하는 것은 CUDA 개발자에게 매우 기본적인 역량으로, 더 많은 고급 메모리 관리 기법에 있어 전제적인 역할을 합니다.

---
## 전제 조건

이 실습을 최대한 잘 활용하려면 여러분은 이미 다음을 하실 수 있어야 합니다.

- CPU 함수를 호출하고 GPU 커널을 실행하는 C/C++ 프로그램 작성, 컴파일, 실행
- 실행 구성을 이용한 병렬 스레드 계층 구조 제어
- 직렬 루프를 GPU에서 병렬로 반복 실행하도록 리팩터링
- 통합 메모리 할당 및 해제

---
## 목표

이 실습을 완료할 때는 다음을 하실 수 있게 됩니다.

- Nsight Systems 명령줄 도구(**nsys**)를 사용하여 가속 애플리케이션 성능 프로파일링
- **스트리밍 멀티프로세서**에 대한 이해를 활용하여 실행 구성 최적화
- 페이지 폴트 및 데이터 마이그레이션에 관한 **통합 메모리** 동작 이해
- 성능 향상을 위해 **비동기 메모리 프리페치**를 사용하여 페이지 폴트 및 데이터 마이그레이션 감소
- 애플리케이션을 빠르게 가속화 및 배포하기 위해 반복 개발 주기 적용

---
## NVIDIA 명령줄 프로파일러를 통한 반복 최적화

가속 코드베이스 최적화 시도가 실제로 성공적이었다는 것을 확인할 수 있는 유일한 방법은 애플리케이션의 성능에 대한 정량적 정보를 위해 애플리케이션을 프로파일링하는 것입니다. `nsys`는 Nsight Systems 명령줄 도구입니다. CUDA 툴킷과 함께 제공되며, 가속 애플리케이션 프로파일링을 위한 강력한 도구입니다.

`nsys`는 사용하기 쉽습니다. 가장 기본적인 사용법은 단순히 `nvcc`로 컴파일된 실행 파일을 경로로 전달하는 것입니다. `nsys`는 애플리케이션이 애플리케이션의 GPU 활동, CUDA API 호출 요약뿐 아니라 이 실습의 후반부에 광범위하게 다루게 될 주제인 **통합 메모리** 활동에 대한 정보를 출력하고 난 뒤 계속해서 애플리케이션을 실행합니다.

애플리케이션을 가속화하거나 이미 가속화된 애플리케이션을 최적화할 때는 과학적이고 반복적인 접근 방식을 취하세요. 변경 사항을 적용한 후 애플리케이션을 프로파일링하고, 메모하고, 성능에 대한 리팩터링의 영향을 기록하세요. 이러한 관찰을 일찍, 자주 하세요. 가속 애플리케이션을 보내는 것 정도의 약간의 노력으로 충분한 성능 향상을 얻을 수 있는 경우가 빈번히 있습니다. 또한, 프로파일링을 자주 하면 CUDA 코드베이스에 대한 특정 변경 사항이 실제 성능에 어떤 영향을 미치는지 배울 수 있습니다. 코드베이스에서 여러 유형의 변경을 거친 후에만 프로파일링을 하는 경우에는 확보하기 어려운 지식이죠.

### 연습문제: nsys로 애플리케이션 프로파일링

[01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) (<------ 이 링크를 포함하여 이 실습의 모든 소스 파일 링크를 클릭하면 열어서 편집할 수 있습니다)는 단순하게 가속화된 벡터 추가 프로그램입니다. 아래 2개의 코드 실행 셀을 사용하세요(`CTRL` + 클릭). 첫 번째 코드 실행 셀은 벡터 추가 프로그램을 컴파일 (및 실행)합니다. 두 번째 코드 실행 셀은 방금 `nsys profile`을 사용하여 컴파일한 실행 파일을 프로파일링합니다.

`nsys profile`은 다양한 방식으로 활용할 수 있는 `qdrep` 보고서 파일을 생성합니다. `--stats=true` 플래그를 사용하여 출력된 요약 통계를 표시하겠습니다. 출력된 정보가 꽤 많습니다.

- 프로파일 구성 세부 정보
- 보고서 파일 세대 세부 정보
- **CUDA API 통계**
- **CUDA 커널 통계**
- **CUDA 메모리 작업 통계(시간 및 크기)**
- OS 런타임 API 통계

이 실습에서는 **굵게** 처리된 3개 섹션을 주로 사용하게 됩니다. 다음 실습에서는 생성된 보고서 파일을 사용하여 Nsight Systems에 비주얼 프로파일링을 위한 GUI를 제공합니다.

애플리케이션을 프로파일링한 후 프로파일링 출력에 표시되는 정보를 사용하여 다음 질문에 답하세요.

- 이 애플리케이션에서 호출된 유일한 CUDA 커널의 이름은 무엇인가요?
- 이 커널은 몇 번 실행되었나요?
- 이 커널을 실행하는 데 얼마나 걸렸나요? 이 시간을 어딘가에 기록해 두세요. 이 애플리케이션을 최적화할 때 얼마나 더 빠르게 만들 수 있는지 궁금하실 테니까요.

In [3]:
!nvcc -o single-thread-vector-add 01-vector-add/01-vector-add.cu -run

Success! All values calculated correctly.


In [4]:
!nsys profile --stats=true ./single-thread-vector-add

Success! All values calculated correctly.
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-2730-b1ba-0219-9d47.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-2730-b1ba-0219-9d47.qdrep"
Exporting 3672 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-2730-b1ba-0219-9d47.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average      Minimum     Maximum            Name         
 -------  ---------------  ---------  ------------  ----------  ----------  ---------------------
    99.1       2350228784          1  2350228784.0  2350228784  2350228784  cudaDeviceSynchronize
     0.8         18403142          3     6134380.7     5495458     7242022  cudaFree             
     0.1          1843570          1     1843570.0     1843570     1843570  cudaLaunchKe

언급하고 넘어갈 만한 것은 기본적으로 `nsys profile`가 기존 보고서 파일을 덮어쓰지 않는다는 점입니다. 이는 프로파일링 시 작업이 실수로 손실되는 것을 방지하기 위해 수행됩니다. 어떤 이유로든, 가령 빠른 반복 작업 중에 기존 보고서 파일을 덮어쓰기를 원한다면, `-f` 플래그를 `nsys profile`에 제공해 기존 보고서 파일 덮어쓰기를 허용할 수 있습니다.

### 연습문제: 최적화 및 프로파일링

1-2분 정도의 시간을 내어 [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu)가 단일 스레드 블록의 많은 스레드에서 실행되도록 실행 구성을 업데이트하여 간단한 최적화를 진행하세요. 다시 컴파일한 다음 아래의 코드 실행 셀을 사용하여 `nsys profile --stats=true`로 프로파일링하세요. 프로파일링 출력을 사용하여 커널의 런타임을 확인하세요. 이 최적화로 속도가 얼마나 향상되었나요? 결과를 어딘가에 기록해 두세요.

In [20]:
!nvcc -o multi-thread-vector-add 01-vector-add/01-vector-add.cu -run

Success! All values calculated correctly.


In [21]:
!nsys profile --stats=true ./multi-thread-vector-add

Success! All values calculated correctly.
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-0fe9-7955-2674-a9d7.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-0fe9-7955-2674-a9d7.qdrep"
Exporting 4147 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-0fe9-7955-2674-a9d7.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average     Minimum    Maximum           Name         
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    55.4        219068724          3   73022908.0      22931  218969821  cudaMallocManaged    
    39.9        157530581          1  157530581.0  157530581  157530581  cudaDeviceSynchronize
     4.7         18634201          3    6211400.3    5551606    7308290  cudaFree             
     

### 연습문제: 반복 최적화

이 연습문제에서는 [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu)의 실행 구성을 편집하고 프로파일링한 뒤 영향을 확인하기 위해 결과를 기록하는 여러 주기를 거치게 됩니다. 작업하는 동안 다음 가이드라인을 활용하세요.

- 실행 구성을 업데이트하는 3~5가지의 다양한 방법을 나열하는 것으로 시작하여 다양한 그리드와 블록 크기 조합을 다루도록 하세요.
- [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) 프로그램을 여러분이 나열한 방법 중 하나로 편집하세요.
- 아래 2개의 코드 실행 셀을 사용하여 업데이트된 코드를 컴파일 및 프로파일링하세요.
- 프로파일링 출력에 나타나는 대로 커널 실행의 런타임을 기록하세요.
- 위에서 가능하다고 나열한 각각의 최적화 방법에 대해 편집/프로파일/기록 주기를 반복하세요.

시도한 실행 구성 중 가장 빠른 것으로 증명된 구성은 무엇인가요?

In [22]:
!nvcc -o iteratively-optimized-vector-add 01-vector-add/01-vector-add.cu -run

Success! All values calculated correctly.


In [23]:
!nsys profile --stats=true ./iteratively-optimized-vector-add

Success! All values calculated correctly.
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-31a6-3fb8-c360-4bd4.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-31a6-3fb8-c360-4bd4.qdrep"
Exporting 5044 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-31a6-3fb8-c360-4bd4.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average     Minimum    Maximum           Name         
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    60.2        225837955          3   75279318.3      17895  225760543  cudaMallocManaged    
    34.7        130224878          1  130224878.0  130224878  130224878  cudaDeviceSynchronize
     5.1         18967389          3    6322463.0    5649094    7495439  cudaFree             
     

---
## 멀티프로세서 스트리밍 및 디바이스 쿼리

이 섹션에서는 GPU 하드웨어의 특정 특징에 대한 이해를 통해 최적화의 수준을 올릴 수 있는 방법을 알아봅니다. **스트리밍 멀티프로세서** 개론을 배운 후, 여러분이 작업 중인 가속 벡터 추가 프로그램을 더욱 최적화해 보게 됩니다.

다음 슬라이드에서는 앞으로 다룰 콘텐츠를 고수준에서 시각적으로 보여드립니다. 다음 섹션으로 넘어가 각 주제를 더 자세히 다루기 전에 슬라이드를 클릭해 보세요.

In [ ]:
%%HTML

<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vRByDOlhmGKNY9IgFonAhE-uM0NAPdZGo8v8vlBBPqRB7RDx-E5g0OnGOpC2VoO-eWFhZBWv5yCtGfk/embed?start=false&loop=false&delayms=3000" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

### 스트리밍 멀티프로세서 및 워프

CUDA 애플리케이션이 실행되는 GPU에는 **스트리밍 멀티프로세서** 또는 **SM**이라고 불리는 처리 단위가 있습니다. 커널이 실행되는 동안 스레드 블록은 실행을 위해 SM에 주어집니다. 가능한 한 많은 병렬 연산을 수행하도록 GPU의 성능을 지원하기 위해, *주어진 GPU에서 SM 수의 배수인 블록 개수를 가지는 그리드 크기를 선택하는 것*으로 성능 향상을 얻을 수 있습니다.

또한 SM은 **워프**라고 불리는 블록 내에서 32개 스레드의 그룹을 생성, 관리, 스케쥴링 및 실행합니다. [SM 및 워프의 심화 내용](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation)을 더 다루는 것은 이 과정의 범위 밖이지만, *스레드 수가 32의 배수인 블록 크기를 선택*함으로써 성능 향상을 얻을 수 있다는 것을 아는 것이 중요합니다.

### 프로그래밍적으로 GPU 디바이스 속성 쿼리하기

GPU의 SM 수는 사용되는 특정 GPU에 따라 다를 수 있으므로, 이동성을 지원하기 위해서는 코드베이스에 SM의 수를 하드 코딩하지 말아야 합니다. 이러한 정보는 프로그래밍적으로 취득해야 합니다.

다음은 CUDA C/C++에서 SM의 수를 포함하여 현재 활성 GPU 디바이스에 대한 여러 속성이 포함된 C 구조체를 얻는 방법을 보여줍니다.

```cpp
int deviceId;
cudaGetDevice(&deviceId);                  // `deviceId` now points to the id of the currently active GPU.

cudaDeviceProp props;
cudaGetDeviceProperties(&props, deviceId); // `props` now has many useful properties about
                                           // the active GPU device.
```

### 연습문제: 디바이스 쿼리

현재 [`01-get-device-properties.cu`](../edit/04-device-properties/01-get-device-properties.cu)에는 수많은 미지정 변수가 포함되어 있으며, 현재 활성 GPU에 대한 세부 정보를 설명하도록 의도된 이런저런 정보를 출력합니다.

[`01-get-device-properties.cu`](../edit/04-device-properties/01-get-device-properties.cu)가 소스 코드에 나타난 원하는 디바이스 속성의 실제 값을 출력하도록 빌드를 만드세요. 여러분의 작업을 지원하고 문서를 소개하는 의미로, [CUDA 런타임 문서](http://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html)를 사용하여 디바이스 속성 구조체의 관련 속성을 파악하는 데 도움을 받으세요. 도중에 막히면 [해답](../edit/04-device-properties/solutions/01-get-device-properties-solution.cu)을 참고하세요.

In [ ]:
!nvcc -o get-device-properties 04-device-properties/01-get-device-properties.cu -run

### 연습문제: SM의 수에 맞는 크기의 그리드를 통해 벡터 추가 최적화

디바이스에 SM의 수를 쿼리하는 능력을 활용해 [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) 내에서 작업 중인 `addVectorsInto` 커널을 리팩터링하여 디바이스 SM 수의 배수와 같은 개수의 블록을 포함하는 그리드에서 실행되도록 하세요.

작성한 코드의 다른 특정 세부 정보에 따라 이 리팩터는 커널의 성능에 개선 또는 현저한 변화를 줄 수도, 주지 않을 수도 있습니다. 따라서, 언제나 그렇듯이, `nsys profile`을 사용하여 성능 변경을 정량적으로 평가할 수 있도록 하세요. 프로파일링 출력 콘텐츠를 기반으로 지금까지의 결과를 나머지 발견들과 함께 기록하세요.

In [ ]:
!nvcc -o sm-optimized-vector-add 01-vector-add/01-vector-add.cu -run

In [ ]:
!nsys profile --stats=true ./sm-optimized-vector-add

---
## 통합 메모리 세부 정보

`cudaMallocManaged`를 사용하여 호스트 또는 디바이스 코드에 사용할 메모리를 할당했는데, 지금까지는 **통합 메모리**(**UM**)가 실제 작업에서 `cudaMallocManaged`에 의해 어떻게 할당되는지에 대한 세부 콘텐츠를 알아보지 않고 자동 메모리 마이그레이션, 프로그래밍 용이성 등 이 방법의 이점을 누렸습니다.

`nsys profile`은 가속 애플리케이션의 UM 관리에 대한 세부 정보를 제공하며, 이 정보를 UM의 작동 방식에 대한 자세한 이해와 함께 사용하여 가속 애플리케이션을 최적화할 추가적인 기회를 제공합니다.

다음 슬라이드에서는 앞으로 다룰 콘텐츠를 고수준에서 시각적으로 보여드립니다. 다음 섹션으로 넘어가 각 주제를 더 자세히 다루기 전에 슬라이드를 클릭해 보세요.

In [ ]:
%%HTML

<div align="center"><iframe src="https://docs.google.com/presentation/d/e/2PACX-1vS0-BCGiWUb82r1RH-4cSRmZjN2vjebqoodlHIN1fvtt1iDh8X8W9WOSlLVxcsY747WVIebw13cDYBO/embed?start=false&loop=false&delayms=3000" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe></div>

### 통합 메모리 마이그레이션

UM이 할당되면, 메모리는 호스트 또는 디바이스에 아직 상주하지 않는 상태입니다. 호스트 또는 디바이스가 메모리에 액세스하려고 시도하면 호스트 또는 디바이스가 필요한 데이터를 배치로 마이그레이션하는 시점에 [페이지 폴트](https://en.wikipedia.org/wiki/Page_fault)가 일어납니다. 이와 유사하게, CPU 또는 가속 시스템의 GPU가 아직 상주하지 않는 메모리에 액세스하려고 시도할 때 페이지 폴트가 발생하고 마이그레이션이 트리거됩니다.

온디맨드 페이지 폴트 및 메모리 마이그레이션 능력은 가속 애플리케이션의 개발 용이성에 엄청난 도움이 됩니다. 또한, 희소한 액세스 패턴을 보이는 데이터로 작업하는 경우(예: 애플리케이션이 실제로 실행될 때까지 어떤 데이터를 처리해야 하는지 알 수 없는 경우) 및 여러 개의 GPU가 탑재된 가속 시스템의 여러 GPU 디바이스에서 데이터에 액세스할 수 있는 시나리오의 경우, 온디맨드 메모리 마이그레이션이 상당히 유용합니다.

예를 들어 런타임 이전에 데이터 요구 사항이 알려져 있고 대규모 연속 메모리 블록이 필요한 경우 페이지 폴트 및 온디맨드 데이터 마이그레이션 오버헤드가 되도록 피하는 것이 좋은 오버헤드 비용을 발생시킬 때가 있습니다.

이 실습의 나머지 콘텐츠 대부분은 온디맨드 마이그레이션을 이해하고 프로파일러의 출력값에서 이를 파악하는 방법을 이해하는 데 할애됩니다. 이 지식을 시나리오에서 유용하게 활용하여 오버헤드를 줄일 수 있습니다.

### 연습문제: UM 마이그레이션 및 페이지 폴트 알아보기

`nsys profile`은 프로파일링된 애플리케이션의 UM 동작을 설명하는 콘텐츠를 출력합니다. 이 연습문제에서는 애플리케이션에서 간단하게 몇 가지를 수정한 다음 `nsys profile` UM 데이터 마이그레이션이 어떻게 동작하는지 살펴봅니다.

[`01-page-faults.cu`](../edit/06-unified-memory-page-faults/01-page-faults.cu)는 `hostFunction`과 `gpuKernel`을 포함하고 있으며, 둘 다 `2<<24` 요소 벡터의 요소를 숫자 `1`로 초기화하는 데 사용할 수 있습니다. 현재는 호스트 함수나 GPU 커널이 사용되고 있지 않습니다.

아래의 4개 질문에 대해 여러분이 방금 UM 동작에 대해 학습한 콘텐츠를 고려할 때 먼저 어떤 유형의 페이지 폴트가 발생해야 하는지 가설을 세운 다음 [`01-page-faults.cu`](../edit/06-unified-memory-page-faults/01-page-faults.cu)를 편집하여 코드베이스에 제공된 2개 함수 중 하나 또는 둘 다를 사용함으로써 시나리오를 생성하고 가설을 테스트해 보세요.

가설을 테스트하기 위해 아래의 코드 실행 셀을 사용하여 코드를 컴파일 및 프로파일링하세요. `nsys profile --stats=true` 출력에서 얻은 결과뿐만 아니라 가설도 기록해야 합니다. 여러분은 `nsys profile --stats=true`의 출력에서 다음을 찾아야 합니다.

- 출력에 _CUDA 메모리 작동 통계_ 섹션이 있나요?
- 있다면 호스트에서 디바이스(HtoD) 마이그레이션과 디바이스에서 호스트(DtoH) 마이그레이션 중 무엇을 나타내나요?
- 마이그레이션이 있는 경우, 출력 콘텐츠에 따르면 몇 번의 _연산_이 이루어졌나요? 작은 메모리 마이그레이션 작업이 많다면 온디맨드 페이지 폴트가 발생하고 있으며, 요청된 위치에 페이지 폴트가 있을 때마다 작은 메모리 마이그레이션이 일어난다는 신호입니다.

여러분이 탐구할 수 있는 시나리오와 도중에 막힐 경우 참고할 수 있는 해답을 드리겠습니다.

- CPU에서만 통합 메모리에 액세스할 때 메모리 마이그레이션 및 페이지 폴트에 대한 증거가 있나요? ([해답](../edit/06-unified-memory-page-faults/solutions/01-page-faults-solution-cpu-only.cu))
- GPU에서만 통합 메모리에 액세스할 때 메모리 마이그레이션 및 페이지 폴트에 대한 증거가 있나요? ([해답](../edit/06-unified-memory-page-faults/solutions/02-page-faults-solution-gpu-only.cu))
- CPU에서 통합 메모리에 먼저 액세스한 다음 GPU에서 액세스하는 경우 메모리 마이그레이션 및 페이지 폴트에 대한 증거가 있나요? ([해답](../edit/06-unified-memory-page-faults/solutions/03-page-faults-solution-cpu-then-gpu.cu))
- GPU에서 통합 메모리에 먼저 액세스한 다음 CPU에서 액세스하는 경우 메모리 마이그레이션 및 페이지 폴트에 대한 증거가 있나요? ([해답](../edit/06-unified-memory-page-faults/solutions/04-page-faults-solution-gpu-then-cpu.cu))

In [ ]:
!nvcc -o page-faults 06-unified-memory-page-faults/01-page-faults.cu -run

In [ ]:
!nsys profile --stats=true ./page-faults

### 연습문제: 벡터 추가 프로그램을 위한 UM 동작 다시 알아보기

이 실습 내내 여러분이 작업 중인 [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) 프로그램으로 돌아가서 현재 상태의 코드베이스를 검토하고, 발생할 것으로 예상되는 메모리 마이그레이션 및 페이지 폴트의 종류에 대한 가설을 세우세요. 마지막 리팩터의 프로파일링 출력을 살펴보고(위로 스크롤하거나 아래에 있는 코드 실행 셀을 실행하여 출력 확인) 프로파일러 출력의 _CUDA 메모리 작동 통계_ 섹션을 관찰하세요. 코드베이스 콘텐츠를 기반으로 마이그레이션의 종류와 연산 수를 설명할 수 있나요?

In [ ]:
!nsys profile --stats=true ./sm-optimized-vector-add

### 연습문제: 커널의 벡터 초기화

`nsys profile`이 커널을 실행하는 데 소요되는 시간을 알려줄 때, 표시되는 실행 시간에는 이 커널의 실행 중에 발생하는 HtoD 페이지 폴트 및 데이터 마이그레이션이 포함됩니다.

이를 염두에 두고 [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) 프로그램에 있는 `initWith` 호스트 함수를 대신 GPU에서 병렬로 할당된 벡터를 초기화하는 CUDA 커널이 되도록 리팩터링하세요. 리팩터링된 애플리케이션을 성공적으로 컴파일 및 실행한 후, 프로파일링하기 전 다음에 대한 가설을 세우세요.

- 리팩터가 UM 메모리 마이그레이션 동작에 어떤 영향을 미칠 것으로 예상되나요?
- 리팩터가 보고된 `addVectorsInto`의 런타임에 어떤 영향을 미칠 것으로 예상되나요?

다시 한 번 말씀드리지만, 결과를 기록해 두세요. 도중에 막히면 [해답](../edit/07-init-in-kernel/solutions/01-vector-add-init-in-kernel-solution.cu)을 참고하세요.

In [ ]:
!nvcc -o initialize-in-kernel 01-vector-add/01-vector-add.cu -run

In [ ]:
!nsys profile --stats=true ./initialize-in-kernel

---
## 비동기 메모리 프리페칭

**비동기 메모리 프리페칭**은 HtoD 및 DtoH 메모리 전송 둘 다에서 페이지 폴트 및 온디맨드 메모리 마이그레이션의 오버헤드를 줄이는 강력한 기법입니다. 프로그래머는 이 기술을 사용하여 애플리케이션 코드로 백그라운드에서 사용 전 시스템 내 모든 CPU 또는 GPU 디바이스로 통합 메모리(UM)를 비동기 마이그레이션할 수 있습니다. 이렇게 하면 페이지 폴트 및 온디맨드 데이터 마이그레이션 오버헤드가 감소하여 GPU 커널 및 CPU 함수 성능이 향상될 수 있습니다.

또한 프리페칭은 데이터를 더 큰 덩어리로 마이그레이션하는 경향이 있으므로 온디맨드 마이그레이션보다 더 적게 이동할 수 있습니다. 따라서 런타임 전에 데이터 액세스 요구 사항을 알 수 있으며 데이터 액세스 패턴이 희소하지 않을 때 탁월한 적합성을 보입니다.

CUDA를 이용하면 `cudaMemPrefetchAsync` 함수를 통해 관리되는 메모리를 GPU 디바이스 또는 CPU에 쉽게 비동기 프리페칭할 수 있습니다. 다음은 현재 활성 GPU 디바이스와 CPU에 데이터를 프리페칭하는 데 이를 사용하는 예입니다.

```cpp
int deviceId;
cudaGetDevice(&deviceId);                                         // The ID of the currently active GPU device.

cudaMemPrefetchAsync(pointerToSomeUMData, size, deviceId);        // Prefetch to GPU device.
cudaMemPrefetchAsync(pointerToSomeUMData, size, cudaCpuDeviceId); // Prefetch to host. `cudaCpuDeviceId` is a
                                                                  // built-in CUDA variable.
```

### 연습문제: 메모리 프리페치

이 시점의 실습에서 여러분의 [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) 프로그램은 CUDA 커널을 실행하여 2개 벡터를 `cudaMallocManaged`로 할당된 세 번째 솔루션 벡터에 추가해야 할 뿐만 아니라, 3개 벡터를 병렬로 각각 CUDA 커널에서 초기화해야 합니다. 어떤 이유로든 애플리케이션이 위의 작업을 하지 않는 경우 [참조 애플리케이션](../edit/08-prefetch/01-vector-add-prefetch.cu)을 참고하셔서 여러분의 코드베이스가 현재 기능을 반영하도록 업데이트하세요.

[01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) 애플리케이션 안의 `cudaMemPrefetchAsync`를 사용하여 세 번의 실험을 진행하며 페이지 폴트 및 메모리 마이그레이션에 미치는 영향을 이해해 보세요.

- 초기화된 벡터 1개를 디바이스에 프리페치하면 어떤 일이 일어나나요?
- 초기화된 벡터 2개를 디바이스에 프리페치하면 어떤 일이 일어나나요?
- 초기화된 벡터 3개 모두를 디바이스에 프리페치하면 어떤 일이 일어나나요?

각 실험 전 UM 동작, 특히 페이지 폴트와 초기화 커널의 보고된 런타임에 미칠 영향에 대한 가설을 세우고, `nsys profile`을 실행하여 확인하세요. 도중에 막히면 [해답](../edit/08-prefetch/solutions/01-vector-add-prefetch-solution.cu)을 참고하세요.

In [ ]:
!nvcc -o prefetch-to-gpu 01-vector-add/01-vector-add.cu -run

In [ ]:
!nsys profile --stats=true ./prefetch-to-gpu

### 연습문제: 메모리를 CPU로 다시 프리페치

`addVectorInto` 커널의 정확도를 확인하는 함수를 위해 다시 CPU로 프리페치를 추가하세요. 다시 한 번, `nsys`에서 프로파일링하여 확인하기 전에 UM에 미칠 영향에 대해 가설을 세우세요. 도중에 막히면 [해답](../edit/08-prefetch/solutions/02-vector-add-prefetch-solution-cpu-also.cu)을 참고하세요.

In [ ]:
!nvcc -o prefetch-to-cpu 01-vector-add/01-vector-add.cu -run

In [ ]:
!nsys profile --stats=true ./prefetch-to-cpu

비동기 프리페칭을 사용하는 일련의 리팩터 후, 횟수는 적지만 크기는 큰 메모리 전송이 일어났으며 커널 실행 시간이 현저히 감소했음을 볼 수 있습니다.

---
## 요약

이 시점의 실습에서는 다음을 할 수 있습니다.

- Nsight Systems 명령줄 도구(**nsys**)를 사용하여 가속 애플리케이션 성능 프로파일링
- **스트리밍 멀티프로세서**에 대한 이해를 활용하여 실행 구성 최적화
- 페이지 폴트 및 데이터 마이그레이션에 관한 **통합 메모리** 동작 이해
- 성능 향상을 위해 **비동기 메모리 프리페치**를 사용하여 페이지 폴트 및 데이터 마이그레이션 감소
- 애플리케이션을 빠르게 가속화 및 배포하기 위해 반복 개발 주기 적용

학습 콘텐츠를 정리하고 여러분의 애플리케이션 가속화, 최적화, 배포 능력을 강화하기 위해 이 실습의 마지막 연습문제를 진행하세요. 이 과정을 완료한 후 시간과 관심이 있으신 분들은 *고급 콘텐츠* 섹션을 진행하시기 바랍니다.

---
## 최종 연습문제: 가속 SAXPY 애플리케이션 반복 최적화

기본적인 가속 [SAXPY](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms#Level_1) 애플리케이션을 [여기](../edit/09-saxpy/01-saxpy.cu)에서 제공합니다. 현재 컴파일, 실행, `nsys profile`을 이용한 프로파일링 전에 찾아 수정해야 하는 몇 가지 버그가 포함되어 있습니다.

버그를 수정하고 애플리케이션을 프로파일링한 후 `saxpy` 커널의 런타임을 기록하고 *반복해서* 애플리케이션 최적화 작업을 하세요. 매 반복이 끝나면 `nsys profile`을 사용하여 코드 변경 콘텐츠가 커널 성능 및 UM 동작에 미친 영향을 확인하세요.

이 실습에서 배운 기법을 활용하세요. 학습을 지원하기 위해 강의 초반에 다룬 기술의 특정 정보를 서둘러 찾아보기보다는 가능할 때마다 [기억 탐색](http://sites.gsu.edu/scholarlyteaching/effortful-retrieval/)을 활용하세요.

여러분의 최종 목표는 `N`을 수정하지 않고 *100us* 이하에서 실행되도록 정확한 `saxpy` 커널을 프로파일링하는 것입니다. 도중에 막힌 경우 [해답](../edit/09-saxpy/solutions/02-saxpy-solution.cu)을 확인하시고, 원하신다면 컴파일 및 프로파일링을 해 보세요.

In [ ]:
!nvcc -o saxpy 09-saxpy/01-saxpy.cu -run

In [ ]:
!nsys profile --stats=true ./saxpy